# oneM2M - Subscriptions and Notifications

This notebook demonstrates how to subscribe to notifications from resources.

<font color="blue">**Please execute the notebook "oneM2M - Subscriptions and Notifications - Notification Server" first from [this notebook](start-notificationServer.ipynb)**</font> .  
This starts a notification server that will receive the notifications that result from the following requests.

- Create a &lt;Subscription> resource
- Create &lt;ContentInstance> resource and raise a notification


## Intitialization
The section does import necessary modules and configurations.

In [ ]:
%run init.py

## Subscribe to a &lt;Container> Resource

The following requests creates a &lt;Subscription> resource for the *myContainer* resource. It

These attributes are part of the request:

- **nu** : The Notification URI specifies where to send verification and notification requests. This points to the Notification Server in the other running notebook (s.a.)
- **nct** : The Notification Content Type attribute specifies the kind of content that should be sent in a notification initiated by this subscription. Possible values are:

| Value | Interpretation                           |
|-------|:-----------------------------------------|
| 1     | All. The whole resource                  |
| 2     | Only modified attributes                 |
| 3     | The resource's *resource ID*             |
| 4     | The trigger payload in case of Triggers  |

- **exc** : The Expiration Counter indicates that the subscriber wants to set the life of this subscription to a limit of a maximum number of notifications.
- **enc/net** : The Notification Event Types indicates the reason for which a notification is to be generated. This can be a list of reasons taken from the following list:

| Value | Interpretation                                        |
|-------|:------------------------------------------------------|
| 1     | Update of Resource                                    |
| 2     | Delete of Resource                                    |
| 3     | Create of Direct Child Resource                       |
| 4     | Delete of Direct Child Resource                       |
| 5     | Retrieve of Container Resource With No Child Resource |
| 6     | Trigger Received For AE_Resource                      |
| 7     | Blocking Update                                       |



In [ ]:
CREATE (                                           # CREATE request
    f'{url}/Notebook-AE/Container',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'X-M2M-RVI'    : '3',                      # Release verson indicator
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=23'  # Content is JSON, and represents a <Subscription> resource
    },
    
    # Request Body
    {
        'm2m:sub': {
            'rn'  : 'mySubscription',
            'nu'  : [ notificationURL ],
            'nct' : 1,
            'enc' : {
                'net': [ 1, 2, 3, 4 ]
            }
        }
    }
)

**Please check the [Notification Server](start-notificationServer.ipynb) notebook** (the one with the running Notification Server) in order to see the verification request. When this requests returns with a *201 (Created)* result code then the creation of the &lt;Subscription> resource as well as the verification request were successfull.

## Add a &lt;ContentInstance> Resource and Raise a Notification

We now have subscribed to notifications whenever the &lt;Container> changes. We can raise a notification by adding a new &lt;ContentInstance> resource to the container:

In [ ]:
CREATE (                                           # CREATE request
    f'{url}/Notebook-AE/Container',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '123',                    # Request identifier
        'X-M2M-RVI'    : '3',                      # Release verson indicator
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=4'   # Content is JSON, and represents an <ContentInstance> resource
    },
    
    # Request Body
    {
        'm2m:cin' : {
            'cnf' : 'text/plain:0',
            'con' : 'Good news, everybody!'
        }
    }
)

This request and result are similar as the previously when adding a &lt;ContentInstance> resource. But when we look at the output in the Notification Server notebook, we see that the notification transported the same result as this request. This way any receiver of a notification can react to a change of a resource without polling a CSE.

Again, **check the Notification Server notebook for the notification**.

&nbsp;